# xgboost 与 lightgbm 原理梳理

## 前言

lightgbm与xgboost的基本原理都是基于加法模型（Additive model）和前向分布算法（forward stagewise algorithm）将多个弱分类器组合成一个强分类器。由于lightgbm是在xgboost算法的基础上，对其在算法复杂度以及对高维度数据处理上的优化。故本文先对xgboost的原理及算法进行梳理，然后具体介绍lightgbm的优化算法。需要注意的是，本文只对算法层面进行了梳理，关于xgboost和lightgbm系统设计层面的内容（比如分块并行，缓存优化等）可以参考原论文，如果有理解不正确的地方，也欢迎指正探讨。

## xgboost


xgboost的构建其实可以拆分成两个问题：  
1. 已知每棵树的结构（即已知每棵树的node feature以及split, threshold等信息），如何计算最优参数$w_j$（j表示叶子节点的index）。由于xgboost的基学习器是回归树，这里的$w_j$其实就是模型文件中每个叶子节点的leaf value。 
2. 如何找到最优的切分点以及切分值，即如何确定最优的树结构  

本文将从第一个问题切入，即如何找到xgboost的最优参数。

### 1. gradient tree boosting

如前文所述，xgboost是一个加法模型，这里给出xgboost的基本模型表达式：  
$$\hat y_i^{(t)}=\sum_{j=1}^{T-1}f_j(x_i)+f_j^{(T)}(x_i)$$  
这里的f(x)表示xgboost的基学习器，即CART回归树。xgboost在目标函数上基于gbdt做了改进和增强，增加了正则项来防止过拟合。xgboost的目标函数如下：  
$$obj^{(t)}=\sum_{i=1}^NL(y_i,\hat y_i^{(t)})+\sum_{j=1}^t\Omega(f_i)$$ 
其中正则项可以表示为：  
$$\Omega(f_t)=\gamma T+\frac{1}{2}\lambda\sum_{j=1}^Tw_j^2$$  
需要注意的是，这里T表示叶子节点的个数，$w_j$表示leaf value, $\gamma$和$\lambda$是超参数。<font color="#dd0000">这里的$\gamma$其实就是平时调参中的reg_alpha，$\lambda$就是reg_lambda。</font><br />
   

传统的梯度下降方式求参不适用于树模型的训练，因为树模型的objective function不是连续的，没有办法对其求导。且上面列出的目标函数没有直观的体现需要求解的叶子节点的leaf value, 这里需要对上述按样本进行遍历的目标函数形式进行改写，将其变为按叶子节点进行遍历的形式。即按将原本的目标函数拆分成每个叶子节点的最小值求解，推导如下：    

$obj^{(t)}=\sum_{j=1}^T\sum_{i\epsilon I_j}L(y_i,\hat y_i^{(t)})+\gamma T+\frac{1}{2}\lambda\sum_{j=1}^Tw_j^2$  
$=\sum_{j=1}^T\sum_{i\epsilon I_j}L(y_i,(\hat y_i^{(t-1)}+\hat y_i^{(t)}))+\gamma T+\frac{1}{2}\lambda\sum_{j=1}^Tw_j^2$  

由于$\hat y_i^{(t)} = w_j$  则有  

$=\sum_{j=1}^T\sum_{i\epsilon I_j}L(y_i,(\hat y_i^{(t-1)}+w_j))+\gamma T+\frac{1}{2}\lambda\sum_{j=1}^Tw_j^2$  
$=\gamma T+\sum_{j=1}^T(\sum_{i\epsilon I_j}L(y_i,(\hat y_i^{(t-1)}+w_j))+\frac{1}{2}\lambda w_j^2)$  

由于xgboost的损失函数可以自定义为任意可导凸函数。这里需要采用泰勒二阶展开得到通用的求解公式，即在不确定具体损失函数的情况下，得到最优参数$w_j$的解析式。泰勒公式的具体形式如下：  
$$f(x)\approx f(x_0)+f^{'}(x_0)(x-x_0)+f^{"}(x_0)(x-x_0)^2$$
这里可以将x看作$x=(\hat y_i^{(t-1)}+w_j)$,$x_0=\hat y_i^{(t-1)}$, 则$w_j=x-x_o$，将原本的损失函数进行泰勒二阶展开，可以得到：  
$$L(y_i,(\hat y_i^{(t-1)}+w_j))=g_iw_j+h_iw_j^2$$
需要注意的是，上述公式是移除掉$L(y_i,\hat y_i^{(t-1)})$后的简化公式，将该式代入原公式有：  

$obj^{(t)}=\gamma T+\sum_{j=1}^T(\sum_{i\epsilon I_j}(g_iw_j+h_iw_j^2)+\frac{1}{2}\lambda w_j^2)$  
$obj^{(t)}=\gamma T+\sum_{j=1}^T(w_j\sum_{i\epsilon I_j}g_i+w_j^2\sum_{i\epsilon I_j}h_i+\frac{1}{2}\lambda w_j^2)$   
$obj^{(t)}=\gamma T+\sum_{j=1}^T(w_j\sum_{i\epsilon I_j}g_i+w_j^2(\sum_{i\epsilon I_j}h_i+\frac{1}{2}\lambda))$   

令$\sum_{i\epsilon I_j}g_i=G_j$, $\sum_{i\epsilon I_j}h_i=H_j$，则有：  
$obj^{(t)}=\gamma T+\sum_{j=1}^T(w_jG_j+\frac{1}{2}w_j^2(H_j+\lambda))$ ，对该式求导，有  
$$w_j=-\frac{G_j}{H_j+\lambda}$$   
将$w_j$代入原objective function,可得目标方程的最小值  
$$min obj^{(t)}\approx \gamma T+\sum_{j=1}^T(-\frac{G_j^2}{H_j+\lambda}+\frac{1}{2}\frac{G_j^2}{H_j+\lambda})=\gamma T+\sum_{j=1}^T(-\frac{1}{2}\frac{G_j^2}{H_j+\lambda})=\gamma T-\frac{1}{2}\sum_{j=1}^T(\frac{G_j^2}{H_j+\lambda})$$  
通过以上的推导，第一个问题可以顺利解决，即在已知树结构的情况下，可以推导出最优的leaf value，接下来需要解决的是如何确定最优的树结构？  

### 2. split findig algorithm

首先一个显而易见的方法就是采用贪心算法的方式遍历所有特征，以及遍历每个特征所有可能的取值，最后选择使得gain最小的切分点，这种方法就是exact greedy algorithm for split finding，在xgboost论文中给出了具体的pseudocode如下： 


- exact greedy algorithm for split finding  
![exact greedy algo.PNG](https://s2.loli.net/2022/09/26/tdPJUsjlSG9qXyp.png)  
上图中的score，就是分裂增益，其公式推导如下：  
$obj_{before}=\gamma -\frac{1}{2}(\frac{G^2}{H+\lambda})$   
$obj_{after}=2\gamma -\frac{1}{2}(\frac{G_L^2}{H_L+\lambda}+\frac{G_R^2}{H_R+\lambda})$   
$GAIN = obj_{before}-obj_{after}=-\gamma -\frac{1}{2}(\frac{(G_L+G_R)^2}{(H_L+H_R)+\lambda})+\frac{1}{2}(\frac{G_L^2}{H_L+\lambda}+\frac{G_R^2}{H_R+\lambda})$    
=$\frac{1}{2}(\frac{G_L^2}{H_L+\lambda}+\frac{G_R^2}{H_R+\lambda}-\frac{(G_L+G_R)^2}{(H_L+H_R)+\lambda})-\gamma$  


- approximate algorithm for split finding   
由于逐步贪心的算法复杂度太高，xgboost论文中提出了一种更为有效的算法：近似算法。近似算法的核心思想就是将连续的特征根据特征分布的分位数切分成离散的buckets，寻找最优切分点时无需再遍历所有不同的特征数值，只需遍历候选的split points,极大的提高了计算效率。具体的pseudocode如下所示。论文中给出了该算法实现的两种模式：全局模式和局部模式。  
![approximate algorithm.PNG](https://s2.loli.net/2022/09/26/GRn9AiOkKdfST7w.png)   
上式中，如何获得candidate split points仍然尚未明晰，论文中提出了一种Weighted Quantile Sketch的方法来获得$S_k$。



### 3. Sparsity-aware Split Finding (缺失值处理)  

现实问题中，特征矩阵往往是稀疏的，存在很多缺失值或者0值。xgboost中提出了sparsity-aware split finding的方法来为这些缺失值分入默认的分支。具体的算法实现如下：  

![sparity aware split finding.png](https://img1.imgtp.com/2022/09/28/xLmpBYFi.png)  

在寻找最优切分点时，会将所有缺失值都分别全部分入左支和右支，最后选择gain最大的方向作为该节点缺失值的默认方向。

### 4. xgboost缺点

通过对xgboost算法实现的梳理，不难发现，其两大主要的缺点：
1. 虽然利用预排序和近似算法可以降低寻找最佳分裂点的计算量，但在节点分裂过程中仍需要遍历数据集；
2. 预排序过程的空间复杂度过高，不仅需要存储特征值，还需要存储特征对应样本的梯度统计值的索引，相当于消耗了两倍的内存。 
 
lightgbm的提出就是为了缓解xgboost的这两大缺点。

## lightgbm

简单来说，lightgbm的两个核心优化思想就是  
1、降低训练数据集的规模  
2、降低特征的数量   
lightgbm分别提出了gradient based one-side sampling(GOSS)和exclusive feature bunding来解决以上两个问题。

### 1. gradient based one-side sampling (主要作用是减少样本数)  

基本思想：样本的梯度大小可以反映样本的权重，如果一个样本的梯度很小，说明该样本的训练误差很小，GOSS算法的核心就是保留梯度大的样本，对梯度小的样本进行随机抽样，这样可以在更加关注训练不足的样本的同时，也不会过多改变原始数据的分布，基本算法如下：

![image.png](https://www.likecs.com/default/index/img?u=aHR0cHM6Ly9waWFuc2hlbi5jb20vaW1hZ2VzLzU4Mi8yZjU2Y2FiNjIzNDAwMTY4MTRjODBiYTQ0ZDA3YmQ3Ni5wbmc=) 

goss算法描述：
 输入：训练数据，迭代步数d，大梯度采样率a，小梯度采样率b，损失函数，决策树
 
 输出：训练好的强学习器
1. goss先根据样本梯度的绝对值，对数据进行降序排序操作。
2. 选取a*100%的样本生成一个大梯度样本的子集。
3. 从剩下的样本集合(1-a)*100%中随机抽取b*(1-a)*100%的样本，生成一个小梯度样本点的集合。
4. 将大梯度样本集和小梯度样本及合并。
5. 计算信息增益时，为了放大采样出的小梯度样本，会给小梯度样本乘以一个常数
6. 使用上述的采样的样本，学习一个新的弱学习器
7. 重复上述6个步骤，直到达到规定的迭代次数或者收敛为止。
  通过这个方法，我们能够更多的关注训练不足的数据上，却不会改变原始数据的分布。

需要注意的是，<font color="#dd0000">这里的权重$g$是一阶导g和二阶导h的乘积。</font><br />

在传统的GBDT中，O表示某个固定叶子节点的训练集，分割特征j的分割点d定义为：<br />
$V_{j|O}(d)=\frac{1}{n_O}(\frac{(\sum_{i\epsilon O:x_{ij} <= d}g_i)^2}{n_{l|O}^j(d)} + \frac{(\sum_{i\epsilon O:x_{ij} >d}g_i)^2}{n_{r|O}^j(d)})$   
其中： <br />
$n_o= \sum[x_i\epsilon O]$(某个固定叶子节点的训练样本的个数)<br />
$n_{l|O}^j(d) = \sum[x_i\epsilon O:x_i<=d]$ (在第j个特征上值小于等于d的样本个数)<br />
$n_{r|O}^j(d) = \sum[x_i\epsilon O:x_i>d]$ (在第j个特征上值大于d的样本个数)<br />
遍历每个特征的每个分裂点，找到$d^*_j = argmax_d V_j(d)$ 并计算最大的信息增益$V_j(d_j^*)$，然后，将数据根据特征$j^*$的分裂点$d_{j^*}$​将数据分到左右子节点。<br />
<br />
而在goss中：<br />
1.首先根据数据的梯度进行降序排序。<br />
2.保留top %a个数据实例，作为数据子集A。<br />
3.对于剩下的数据的实例集合$A^c$，随机采样获得大小为$b*|A^c|$的数据子集B。<br />
4.最后我们在集合$A\cup B$上，通过以下方程估计信息增益$\tilde V_j(d)$:<br />

$\tilde V_j(d) = \frac{1}{n}(\frac{(\sum_{i\epsilon A_l}g_i +\frac{1-a}{b}\sum_{i\epsilon b_l}g_i )^2}{n_{l}^j(d)} + \frac{(\sum_{i\epsilon A_r}g_i +\frac{1-a}{b}\sum_{i\epsilon B_r}g_i)^2}{n_{r}^j(d)})$  <br />
$A_l = \{x_i\epsilon {A} : x_{ij} <= d\}$<br />
$A_r = \{x_i\epsilon {A} : x_{ij} > d\}$<br />
$B_l = \{x_i\epsilon {B} : x_{ij} <= d\}$<br />
$B_r = \{x_i\epsilon {B} : x_{ij} > d\}$<br />
这里采样后的数据子集B需要乘上$\frac{1-a}{b}$，是因为采用后样本数量减少，因此需要将B集中的样本权重放大$\frac{1-a}{b}$，使采样后的小梯度样本的梯度和恢复到原来小梯度样本的梯度总和。



### 2. Exclusive Feature Bunding（主要作用是减少特征数）

lightgbm的降维思想是将彼此互斥的特征组成一个bundle,并在直方图构建的过程中，不再是为每个单个特征构建直方图，而是直接为feature bundle构建直方图，这样直方图构建的复杂度就从$O(data \times feature)$ 降为 
$O(data \times bundle)$。这里需要解决两个问题：①决定哪些特征可以绑定为特征束②怎样建立一个特征束。  

对于问题①，lightgbm论文中已经论述这是一个NP-HARD问题，在多项式时间内不可能去找到准确的解决办法。所以这里使用的是一种近似的解决办法，即特征之间允许存在少数的样本点并不是互斥的，允许小部分的冲突可以得到更小的特征绑定数量，更进一步的提高了计算的有效性。  
对于问题②，我们需要一个方法将一个特征束中的特征合并为一个特征，关键就是确保原始特征的值都可以在特征束中识别出。因为基于直方图的算法保存离散的箱值而不是特征的连续值，因此我们可以通过使互斥特征留在不同的bin中来建立特征束。具体地说，就是通过向特征的原始值中增加偏移来完成。  

论文中分别提出了greedy bundling和merge exclusive features来解决这两个问题，算法的pseudocode如下图所示：  


![image.png](https://img2020.cnblogs.com/blog/772554/202004/772554-20200422224239131-799645895.png)

生成特征束的伪代码如Algorithm3所示，核心思想是可以将特征与特征之间的互斥性看作是有权的无向图，其中可以将features看作是图的nodes，edges的权重则是两个特征之间的total conflicts。无向图构建完成后，算法会根据nodes的degree对特征进行排序，第一个for循环遍历所有特征定点第二个for循环遍历当前所有的特征束，计算当前特征加入特征束[j]后特征束[j]的总冲突，如果小于阈值，则将当前特征加入第j个特征束，如果找不到可以加入的特征束，则当前特征形成一个新的特征束。论文中基于algorithm 3又提出了一种优化方法，即不再构建无向图，而是直接基于非零值的个数对特征进行排序，其他方面与算法3相同。<br />

<br />
上面的算法已经划分好了特征束，现在要将一个特征束中的特征进行合并成一个特征。merge exclusive features的基本原理是将互斥的特征通过添加偏移量的方式放到不同的bins里面。 论文中给出了一个具体的例子说明，比如AB特征在一个特征束中，A和B，开始时A=[0,10)，B=[0,20)，当我们在特征B中增加偏移值为10的话，则特征B的取值范围变为[10,30)。之后，将特征A与特征B的值合并到一起，再将特征束的取值范围设置为[0,30)，取代原本的特征A和特征B，如下述Algorithm 4的伪代码所示。下图是关于该算法的一个直观体现：
  
![merge exclusive features.png](https://img1.imgtp.com/2022/09/28/hqNRRfEO.png)

### 3. Histogram算法：

其基本思想是是将连续的浮点特征值进行离散化为k个整数并构造一个宽度为k的直方图。<br />
对某个特征数据进行遍历的时候，将离散化后的值用为索引作为直方图的累积统计量。<br />
遍历完一次后，直方图便可累积对应的统计量，然后根据该直方图寻找最佳分裂点。


![Historgram.png](https://www.freesion.com/images/391/2f5406f9e446c04e48156b20e0416cb7.png)

对于类别特征，按照每类的$\frac{G}{H} = \frac{\sum{gradiend}}{\sum{hessian}}$进行排序，然后按照这个顺序构造直方图，寻找最优分裂点